# 闲话python 48: C/C++扩展Python与Swig工具

python作为一种通用的编程语言，一般而言，是能够满足逻辑实现的需求的。只是在日常使用过程中，除了实现一些逻辑之外，至少还有两个方面的需求是可能需要寻求其他语言帮助的，第一个是提升运行效率，第二个是复用已有C/C++代码。python比较接近自然语言这一特性确实对使用者而言很不错，但是这带来了一个不良后果--运行速度慢。有时还需要借助多进程的方式提升运算速度，但是无论如何，python本身速度慢这件事确实改变不了，也就意味着硬件资源的利用率低。而C/C++的运行速度上的优势让我们比较愿意寻求某种方式在python中调用C/C++的程序。此外，如果并不是白手起家，而是已有大量C/C++代码实现的核心功能，那么使用python重新实现一遍是不划算的。这时将这些C/C++代码编译成python可调用的库将非常具有吸引力。本文将使用三种方式将一个简单的C程序逻辑封装成python中可调用的形式，以展示python使用C/C++扩展功能的一般方式。分别是：动态共享库调用、C扩展以及swig工具。本文的所有演示在MacOS 10.14.6操作系统中完成，Linux操作系统下应该比较容易复现，Windows可能会比较麻烦一点。

## 1. 动态共享库调用

将C/C++代码编译成动态共享库然后在python中调用是一种比较直观的方式。在MacOS和Linux中是生成后缀为so的文件，在windows系统成是生成后缀为dll的文件。首先展示一下这个简单功能的C程序代码。

In [1]:
!cat code/c/dll_example.c


int add_vector(int * p_a, int * p_b, int * p_c, int len){
    if (len <= 0) {
        return 0;
    }
    for (int i=0; i<len; ++i) {
        p_c[i] = p_a[i] + p_b[i];
    }
    return len;
}


这里，我们使用gcc编译器将源代码编译成动态共享库。查看编译产出，发现按照预期生成了so文件。

In [4]:
!gcc -c -fPIC code/c/dll_example.c -o code/c/dll_example.o
!gcc -shared code/c/dll_example.o -o code/c/dll_example.so
!tree code/c

code/c
├── dll_example.c
├── dll_example.o
└── dll_example.so

0 directories, 3 files


接着，就可以在python中加载生成的so文件，然后调用对应的功能函数。不过还是有一点需要说明，即数据类型。使用过C/C++的同学都知道，C/C++是强类型的静态语言，也就是说在执行功能函数的时候，常常必须要保证传参的数据类型完全一致，否则程序容易崩溃。python提供了一个库ctypes来解决调用C/C++程序时的类型问题。通常，我们需要使用ctypes中提供的数据类型和转换接口，将python中的数据转换成C/C++程序中的数据类型，然后调用对应的功能函数。复杂的函数参数对于这种调用方式而言是灾难性的，因此这种方式一般适用于python与C/C++交互接口简单的情形。

In [9]:
import ctypes
import os
# 载入动态链接库
dll_example = ctypes.cdll.LoadLibrary(
    os.path.join(os.getcwd(), 'code/c/dll_example.so'))
# 构造输出数据数据存储区
IntArray5v = ctypes.c_int * 5
pa = IntArray5v(1,2,3,4,5)
pb = IntArray5v(5,4,3,2,1)
pc = IntArray5v(0,0,0,0,0)
# 调用动态链接库中的函数
ret = dll_example.add_vector(pa, pb, pc, len(pa))
# 打印返回结果
print('ret={}'.format(ret))
print('result={}'.format(list(pc)))

ret=5
result=[6, 6, 6, 6, 6]


## 2. C扩展

为了解决上文中调用动态共享库所面临的问题，还可以使用C扩展的方式完成在python中调用C/C++程序的需求。这种方式使用了python本身提供的一组C/C++语言交互接口API，这样所做成的扩展程序并不需要像动态共享库那样显式加载文件，而是像一般的python模块那样导入即可。编程风格更加pythonic，能够实现的交互接口也更加复杂。下面就展示一下使用C扩展的方式需要提供的C程序源代码。

In [2]:
!cat code/c/cext_example.c

#include "Python.h"

int add_vector(int * p_a, int * p_b, int * p_c, int len){
    if (len <= 0) {
        return 0;
    }
    for (int i=0; i<len; ++i) {
        p_c[i] = p_a[i] + p_b[i];
    }
    return len;
}

// 包装python调用接口
static PyObject * cext_add_vector(PyObject *self, PyObject *args){
    PyObject * py_list_a = NULL;
    PyObject * py_list_b = NULL;
    PyObject * py_list_c = NULL;
    if (!PyArg_ParseTuple(args, "OOO", &py_list_a, &py_list_b, &py_list_c)) {
        return NULL;
    }
    int len = PyList_Size(py_list_a);
    int * pa = (int *)malloc(sizeof(int)*len);
    int * pb = (int *)malloc(sizeof(int)*len);
    int * pc = (int *)malloc(sizeof(int)*len);
    int count = len;
    while(count--){
        if(!PyArg_Parse(PyList_GetItem(py_list_a, count), "i", pa+count)){return NULL;}
        if(!PyArg_Parse(PyList_GetItem(py_list_b, count), "i", pb+count)){return NULL;}
    }
    int ret = add_vector(pa, pb, pc, len);
    count = len;
    while(count--){
        PyList_Se

C代码中除了本次演示功能的核心之外，最主要的就是定义调用接口。正是由于在C代码中实现了一个复杂解析过程，python中调用过程就可以非常简单，与一般的python程序调用毫无差别。
C代码编写完成后还需要编写一个对应的setup.py文件，以便于编译。下面展示这个setup.py文件。

In [2]:
!cat code/c/setup.py

from distutils.core import setup, Extension
MOD = 'cext_example'
setup(name=MOD, ext_modules=[Extension(MOD, sources=['cext_example.c'])])


C代码和setup.py文件准备完毕之后就可以进行编译了。下面展示编译的指令，并显示编译产出的文件。

In [13]:
!cd code/c && python3 setup.py build_ext --inplace
!tree code/c | grep "cext_example"

running build_ext
│       ├── cext_example.o
├── cext_example.c
├── cext_example.cpython-37m-darwin.so


可以看到，编译产出顺利生成，接下来就可以在python中调用了。由于C扩展的so文件并不在当前的搜索目录下，因此需要修改一下sys.path这个变量。除了执行核心的功能函数之外，还可以查看模块的文档，如果需要也可以设置对应的版本号。这样就与一般的python模块一致了，用起来也比较顺手。

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'code/c'))
import cext_example
# 查看模块文档
print('doc of module:{}'.format(cext_example.__doc__))
# 设置存储空间
al = [1,2,3,4,5]
bl = [5,4,3,2,1]
cl = [0]*5
# 调用功能函数
ret = cext_example.add_vector(al, bl, cl)
# 查看结果
print('ret={}'.format(ret))
print('result={}'.format(cl))

doc of module:just test for c extension
ret=5
result=[6, 6, 6, 6, 6]


## 3. Swig工具

从上文的C扩展可以看出，C/C++的源码中需要包含python交互的解析，因此显得非常复杂。而这个过程似乎是形式化的，那么有没有什么工具可以辅助完成这个过程呢？这就是这里提到的swig工具。当然，好可以用boost中的wrap接口，但是boost本身比较沉重，在发布和共享时不太方便。下面我们先看一下实现所需功能的C/C++源代码。

In [8]:
!cat code/c/swig_example.c

#include "swig_example.h"

int add_vector(int * p_a, int * p_b, int * p_c, int len){
    if (len <= 0) {
        return 0;
    }
    for (int i=0; i<len; ++i) {
        p_c[i] = p_a[i] + p_b[i];
    }
    return len;
}


In [9]:
!cat code/c/swig_example.h

#ifndef __CEXT_EXAMPLE__H__
#define __CEXT_EXAMPLE__H__

int add_vector(int * p_a, int * p_b, int * p_c, int len);

#endif

然后，需要定义一个后缀为i的描述文件，用于swig工具处理源代码。由于本文所演示的功能中需要使用int类型的数组，所以在描述文件中也添加了一个相关的定义，就可以在python中定义int数组，便于调用。

In [10]:
!cat code/c/swig_example.i

%module swig_example


%{
#include "swig_example.h"
%}

%include "carrays.i"
%array_class(int, intp);

int add_vector(int * p_a, int * p_b, int * p_c, int len);

使用swig指令调用上面所编写的描述文件，就可以生成针对源码的wrap接口代码。从以下的演示可以看出，这条指令生成了swig_example.py和swig_example_wrap.c两个文件。

In [7]:
!cd code/c && swig -python swig_example.i
!tree code/c | grep "swig_example"

├── swig_example.c
├── swig_example.h
├── swig_example.i
├── swig_example.py
└── swig_example_wrap.c


接下来的步骤就跟上文中的C扩展步骤一致了，编写setup.py文件，然后编译。下面展示一个详细一点的setup.py文件，其中可以指定很多与模块相关的信息，便于形成标准的python第三方库。

In [11]:
!cat code/c/swig_setup.py

from distutils.core import setup, Extension

setup(
    name='swig_example',
    version='0.0.1',
    author='Blue Geek',
    description="example for swig python",
    ext_modules=[
        Extension(
            '_swig_example', 
            sources=['swig_example_wrap.c', 'swig_example.c'])],
    py_modules=['swig_example']
)

编译生成对应的so文件之后，就可以在python程序中调用了。

In [12]:
!cd code/c && python3 swig_setup.py build_ext --inplace
!tree code/c | grep "swig_example"

running build_ext
│   └── swig_example.cpython-37.pyc
├── _swig_example.cpython-37m-darwin.so
│       ├── swig_example.o
│       └── swig_example_wrap.o
├── swig_example.c
├── swig_example.h
├── swig_example.i
├── swig_example.py
├── swig_example_wrap.c


同样的，需要先设置以下搜索路径，以便于找到该模块。这里遇到的一个问题是需要保持python调用C/C++函数的传入参数与程序中定义的参数类型一致。由于本文所演示的例子中使用了int类型的指针，那么在描述文件中定义的intp就派上了用场。下面演示了参数转换和调用的过程。但从这个调用来看，似乎并不简单，其原因是参数类型的转换代码较多，而且还是只能服务于单一的数据类型转换。但这并不是一个大问题。通常在设计C/C++程序时，会提供大量简单易用的类型转换的接口，在python中也会封装一些接口，从而避免在实际调用过程中频繁编写代码进行转换。在封装完类型转换接口之后，其调用形式与C/C++源码中定义的是一致的，因此我们并不需要专门地阅读swig所生成的代码，只需要掌握一点swig描述配置即可。

In [7]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'code/c'))
import swig_example

# 从list到c数组的转换
def set_intp(vlist):
    tmp = swig_example.intp(len(vlist))
    for idx,v in enumerate(vlist):
        tmp[idx] = v
    return tmp

# 从c数组到list的转换
def get_value_from_intp(intp, length):
    vlist = []
    for idx, v in enumerate(intp):
        if idx>=length:
            break
        vlist.append(v)
    return vlist
# 创建三个变量
al = set_intp([1,2,3,4,5])
bl = set_intp([5,4,3,2,1])
cl = set_intp([0,0,0,0,0])
# 执行计算任务
ret = swig_example.add_vector(al, bl, cl, 5)
# 查看返回值
print('ret={}'.format(ret))
print('result={}'.format(get_value_from_intp(cl, 5)))

ret=5
result=[6, 6, 6, 6, 6]


到此，使用C/C++扩展python就讨论完毕。上面所描述的三种方法各有利弊，一般而言，交互接口简单的就直接使用动态共享库完成，如果调用接口比较复杂那么可以使用C扩展的方式完成，如果需要封装的C代码比较复杂，那么可以考虑使用swig工具实现，然后配合一些类型转换接口。从实际的开发经验来看，扩展python还是属于比较高阶的用法，一般的日常开发中是很少用到的。在图像处理领域，由于过去曾经积累了大量的C/C++代码，而且对计算效率的要求比较高，相对而言可能需要使用扩展python的地方稍多一点。本文的notebook版本文件在github的cnbluegeek/notebook仓库中共享，欢迎感兴趣的朋友前往下载。